In [1]:
import sqlite3
import pandas as pd

pd.set_option('display.max_rows', 100)

In [2]:
con = sqlite3.connect(':memory:')

In [3]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [4]:
for table in table_names:
    data = pd.read_csv(f'data/{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [5]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

,CategoryID,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"
5,6,Meat/Poultry,Prepared meats
6,7,Produce,Dried fruit and bean curd
7,8,Seafood,Seaweed and fish


In [6]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [7]:
tasks+=1
pd.read_sql(
    '''
    SELECT CustomerName, Country, Address FROM Customers 
    WHERE Country in ('Germany', 'France') or City = 'Madrid'
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [8]:
tasks+=1
pd.read_sql(
    '''
    SELECT Country, count(*) 'Customers count' FROM Customers 
    GROUP BY Country order by count(*) desc limit 3
    ''',
    con,
)

,Country,Customers count
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [9]:
tasks+=1
pd.read_sql(
    '''
    SELECT s.ShipperName, r.OrderDate FROM Shippers s 
    JOIN (SELECT OrderDate, ShipperID FROM Orders order by OrderDate LIMIT 1 OFFSET 9) r
    ON s.ShipperId = r.ShipperId
    ''',
    con,
)

,ShipperName,OrderDate
0,Federal Shipping,02.10.1996


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [10]:
pd.read_sql(
    '''
    SELECT max(cnt) FROM (SELECT count(SupplierId) cnt FROM Products group by ProductId)
    ''',
    con,
)
# Замечание:
# видно, что каждый вид товара доставляется только одним поставщиком, что упрощает дальнейшие запросы

,max(cnt)
0,1


In [11]:
tasks+=1
pd.read_sql(
    '''
    WITH cte AS (
      SELECT d.OrderID, d.Quantity * p.Price as Cost, p.ProductName FROM OrderDetails d 
      JOIN Products p ON d.ProductId = p.ProductId
    )
    SELECT OrderID, ProductName, Cost FROM cte 
    WHERE OrderID = (SELECT OrderID FROM cte group by OrderID order by SUM(Cost) desc LIMIT 1)
    ''',
    con,
)

,OrderID,ProductName,Cost
0,10372,Sir Rodney's Marmalade,972.0
1,10372,Côte de Blaye,10540.0
2,10372,Camembert Pierrot,2380.0
3,10372,Mozzarella di Giovanni,1461.6


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [12]:
tasks+=1
pd.read_sql(
    '''
    SELECT p.ProductName, x.OrderID, x.Qnt FROM Products p
    JOIN (
        SELECT OrderID, ProductId, sum(Quantity) Qnt FROM OrderDetails 
        WHERE ProductId = (
            SELECT ProductId FROM OrderDetails 
            GROUP BY ProductId ORDER BY sum(Quantity) desc LIMIT 1)
        GROUP BY OrderID, ProductID) x 
    ON p.ProductID = x.ProductID
    ''',
    con,
)

,ProductName,OrderID,Qnt
0,Gorgonzola Telino,10253,20
1,Gorgonzola Telino,10272,40
2,Gorgonzola Telino,10273,15
3,Gorgonzola Telino,10325,4
4,Gorgonzola Telino,10335,25
5,Gorgonzola Telino,10342,56
6,Gorgonzola Telino,10356,30
7,Gorgonzola Telino,10359,70
8,Gorgonzola Telino,10363,20
9,Gorgonzola Telino,10374,30


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [13]:
# замечание из 4-го задания про уникального поставщика для каждого продукта упрощает запрос
tasks+=1
pd.read_sql(
    '''
    WITH cte as (
        SELECT p.SupplierID FROM OrderDetails d
        JOIN Products p ON d.ProductID = p.ProductID
        GROUP BY p.SupplierID ORDER BY count(d.OrderID) desc LIMIT 5
    )
    SELECT SupplierName, Country, ContactName, Phone FROM Suppliers 
    WHERE SupplierID IN (SELECT SupplierID FROM cte)
    ''',

    con,
)

,SupplierName,Country,ContactName,Phone
0,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
1,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
2,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755
3,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323
4,Norske Meierier,Norway,Beate Vileid,(0)2-953010


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [14]:
tasks+=1
pd.read_sql(
    '''
    WITH cte as (
        SELECT d.OrderID, d.Quantity * p.Price Cost, p.CategoryID FROM OrderDetails d
        JOIN Products p ON d.ProductID = p.ProductID
    )
    SELECT c.Country, t.CategoryName, sum(cte.Cost) Total FROM cte
    JOIN Orders r ON cte.OrderID = r.OrderID
    JOIN Customers c ON r.CustomerID = c.CustomerID
    JOIN Categories t ON cte.CategoryID = t.CategoryID
    WHERE c.Country = 'Brazil'
    GROUP BY t.CategoryID, t.CategoryName, c.Country
    ORDER BY Total desc LIMIT 1
    ''',
    con,
)

# OrderDetails: OrderID & ProductID & Quantity
# Products: ProductID & Price & CategoryID
# Orders: OrderId & CustomerID
# Customers: CustomerId & Country
# Categories: CategoryID & CategoryName

,Country,CategoryName,Total
0,Brazil,Beverages,13690.0


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [15]:
tasks+=1
pd.read_sql(
    '''
    WITH cte as (
        SELECT d.OrderID, sum(d.Quantity * p.Price) Total FROM OrderDetails d
        JOIN Products p ON d.ProductID = p.ProductID
        GROUP BY d.OrderID
    )
    SELECT max(Total) - min(Total) Diff FROM (
        SELECT cte.OrderID, Total FROM cte
        JOIN Orders r ON cte.OrderID = r.OrderID
        JOIN Customers c ON r.CustomerID = c.CustomerID
        WHERE c.Country = 'USA')
    ''',
    con,
)

# OrderDetails: OrderID & ProductID & Quantity
# Products: ProductID & Price
# Orders: CustomerID & OrderID
# Customers: CustomerID & Country

,Diff
0,7638.45


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [16]:
tasks+=1
pd.read_sql(
    '''
    WITH cte AS (
        SELECT EmployeeID, FirstName || ' ' || LastName Name, 
            date(substr(BirthDate, 7, 4)||'-'||substr(BirthDate, 4, 2)||'-'||substr(BirthDate, 1, 2)) BirthDate 
        FROM Employees
        ORDER BY BirthDate desc
        LIMIT 3
    )
    SELECT Cnt, Name FROM (SELECT EmployeeID, count(OrderID) Cnt FROM Orders GROUP BY EmployeeID) r
    JOIN cte ON r.EmployeeID = cte.EmployeeID
    ''',
    con,
)

,Cnt,Name
0,6,Anne Dodsworth
1,29,Nancy Davolio
2,31,Janet Leverling


10. Сколько банок крабового мяса всего было заказано.

In [17]:
pd.read_sql(
    '''
    SELECT ProductID, ProductName, Unit FROM Products
    WHERE LOWER(ProductName) like '%crab%' 
    ''',
    con,
)
# нашли то что нужно, запомним ProductId (40) и количество банок в упаковке (24)

,ProductID,ProductName,Unit
0,40,Boston Crab Meat,24 - 4 oz tins


In [18]:
tasks+=1
pd.read_sql(
    '''
    SELECT sum(Quantity) * 24 Total FROM OrderDetails WHERE ProductID = 40
    ''',
    con,
)

,Total
0,6144


In [19]:
if tasks==10:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
